# How to test

In [ ]:
# see https://untitled-ai.slack.com/archives/CTA2NC0G7/p1618526841016100 for more details
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

You can run tests in the shell of your experiment notebook by adding computronium to your python path.
```
cd standalone/ballworld
PYTHONPATH=.:../../computronium pytest ...
```

## Read the bottom of this notebook at least once
## The remainder of the notebook is all of the ways to run tests

# The best way to run tests after many changes

--stepwise (can be abbreviated --sw) means:

"run until first failure, and when re-running, begin at the last failure"

-m selects tests via a "mark", ie, just that type of test

Once the integration tests start succeeding, you may wish to run just the last line.

In [ ]:
!pytest -s --stepwise --without-integration --without-slow-integration && \
pytest -s --stepwise -m "integration_test" && \
pytest -s --stepwise -m "slow_integration_test"

# The above, but without displaying test stdout

In [ ]:
!pytest --stepwise --without-integration --without-slow-integration && \
pytest --stepwise -m "integration_test" && \
pytest --stepwise -m "slow_integration_test"

# To run tests in parallel

Add the `-n <NUM_WORKERS>` command when you use `pytest`.

Be careful about running tests that will use the GPU with multiple workers. Tests marked with `@slow_integration` or `@integration`
can run tests on the GPU.

In [ ]:
!pytest --stepwise -n 64 --without-integration --without-slow-integration --durations=0 && \
pytest --stepwise -m "integration_test" --durations=0 && \
pytest --stepwise -m "slow_integration_test" --durations=0

# To run a specific test

The matching is done by string--can be module name, test name, etc

In [ ]:
!pytest -s -k "test_check_stable"

# To debug in pycharm

This flag can be added to any other test.

In order for this to work, you need to start your pycharm remote debug setup **before** you run this command!

In [ ]:
!pytest -s -k "test_check_stable" --pycharm

# To debug in ipdb

This flag can be added to any other test.

This is particularly useful if you're lazy. I'm not sure if it will work well from the jupyter notebook--you may need to run this from the terminal.

In [ ]:
!pytest -s -k "test_check_stable" --pdbcls=IPython.terminal.debugger:TerminalPdb --pdb

# Scratchpad

A place for you to run whatever command you want

In [ ]:
!pytest

# Important information about tests

## Properly setting up paths and env
While the notebook contains the correct python path, you'll need to run this in your terminal to set it correctly:
```
export PYTHONPATH=.
```
You'll also obviously want to change directories to be in your 'current' directory.

## How to create new tests

Simply define any function with a name that starts with `test_` in a python file that either starts with `test_` or ends with `_test.py`.

## Naming conventions

Tests of a single module named `foo.py` should be located in `foo_test.py`.

Higher level tests of multiple modules together should be placed in a file called `test_whatever.py`, where "whatever" should describe what you're trying to test.

## How to use pytest fixtures

Seriously, please go read this (yes the whole thing): https://docs.pytest.org/en/latest/fixture.html

## How we prevent pytest fixtures from being totally unmaintainable

pytest fixtures are really great in a lot of ways. However, one way in which they are not great is that they are kind of magical--they rely on the name of a parameter to a function in order to dynamically figure out which fixture to use, with all sorts of insane possibilities.

We restrict ourselves (by convention) from using some of the crazy name overriding that is possible with pytest.

Additionally, we've created our own little wrappers that make it possible to be explicit (per the zen of python) about which fixtures we are using.

Use our **`@fixture`** annotation rather than the pytest one. Note that in order for it to work, you'll want to make the fixture name have an extra `_` at the end--this prevents it from shadowing the name of the variable in the function (which should **not** have that trailing `_`)

The **`@use`** annotation indicates which fixtures should be used in the test. In the future, we will likely be able to autogenerate this annotation. For now, you need to keep it in-sync with the function definition.

## Test style

Please try to keep the duplication of code between tests as low as possible by using pytest fixtures, ideally defined locally, to specify the common resources that should be created and provided to the tests.


## There are 3 types of tests

1. **unit**: These should run extremely quickly (< 0.5 seconds). They should only really test a single module. Their file names should end in `_test.py`, and their names should be synchronized with the name of the file that they are testing. They should not use any external resources (processes, sockets, etc). Temp directories are ok.

2. **integration**: These should run relatively quickly (0.1 second - 10 seconds). They can use any resources (but obviously like any test need to clean up after themselves and be safe from a concurrency perspective). They can also involve more than a single module. This is a good fit for something like "check that godot can create some small number of images".

3. **slow integration**: These are much like the integration tests, but are allowed to take up to 100 seconds. This is a good fit for something like "run many godot processes in parallel and soak test some particular thing". Please try to not to make these. There's no guarantee that they will be run very often because they're so annoyingly slow.

In order to define which type of test a given test is, use the `@integration_test` and `@slow_integration_test` markers from `common.testing_utils`. If unmarked, the test is assumed to be a unit test.

## Notes on output formatting

tqdm works well in notebooks and in the terminal, but if running pytest via a notebook there is no way for it to properly update the output progress bars due to the way that input is captured by pytest and sent to the notebook (because it is not a tty). Thus it is disabled for subprocesses, which would otherwise cause an insane amount of log spam.

## mypy speed

You might think "ugh mypy is so slow", but it actually does a good job of caching, so the *second* time you run it, it will be quite fast. That's why it's a unit test. However, for it to actually be fast, it needs to by run from the same directory, as that is where it creates its `.mypy_cache` directory.

## About this notebook

Because this notebook is synced from your local environment to the notebook machine, but is **not** synced back (unless you run an explicit scp command yourself), it is safe to edit this notebook and put in concrete values for whatever test you want to run, and even hit save. Note that when you next sync your local copy of this to the notebook machine and reload your browser, your changes will be gone.